# Capstone Project Week 3C
Week 3 assignment part C.  
Author: E. Hagen

In [1]:
import numpy as np
import pandas as pd
import requests
import matplotlib
import matplotlib.pyplot as plt

Load wikipedia table into `pandas.DataFrame`:

In [2]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
df = pd.read_html(url, header=0)[0]
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


Remove rows where `'Borough' == 'Not assigned'`:

In [3]:
df.drop(np.where(df['Borough'] == 'Not assigned')[0], inplace=True)
df.reset_index(drop=True, inplace=True)
df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


Set rows where `Neighbourhood == 'Not assigned'` to `Borough` value:

In [4]:
inds = df['Neighbourhood'] == 'Not assigned'
df.loc[inds, ('Neighbourhood')] = df['Borough'][inds]
df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


Concatenate `Neighbourhood` values by their unique `Postcode` and `Borough`:

In [5]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


Print out number of rows in dataset:

In [6]:
print('Number of rows in dataset: {}'.format(df.shape[0]))

Number of rows in dataset: 103


Extract latitude and longitudes for each Neighbourhood, and fill in DataFrame:

In [7]:
# download data
!wget --content-disposition https://cocl.us/Geospatial_data

--2020-02-07 14:02:48--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 159.8.72.228, 159.8.69.21, 159.8.69.24
Connecting to cocl.us (cocl.us)|159.8.72.228|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-02-07 14:02:51--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-02-07 14:02:51--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [

In [8]:
# coordinates for each code
coords = pd.read_csv('Geospatial_Coordinates.csv')
coords

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [9]:
# concatenate location coordinates to DataFrame sorted according to Postcode value 
# (order may not be the same)
df = pd.concat([df, 
                pd.concat([coords.loc[coords['Postal Code']==pc,
                                      ['Latitude', 'Longitude']] 
                           for pc in df.loc[:, 'Postcode']])], 
               axis=1, sort=False)
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


Folium map with location markers for Neighbourhoods with unique postcode:

In [10]:
import folium

In [11]:
# create map of Toronto using latitude and longitude values
toronto_map = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map) 
    
toronto_map

Cluster neighbourhoods based on carthesian coordinates using KMeans++:

In [12]:
from sklearn.cluster import KMeans

# number of clusters
kclusters = 8

# coordinates
coords = df[['Latitude', 'Longitude']]

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(coords)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 5, 2, 5], dtype=int32)

In [13]:
# insert ClusterLabel column in DataFrame
df['ClusterLabel'] = kmeans.labels_
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude,ClusterLabel
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,2
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,2
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,2
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,2
...,...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188,4
99,M9P,Etobicoke,Westmount,43.696319,-79.532242,4
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724,4
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437,4


In [14]:
toronto_map = folium.Map(location=[df['Latitude'].mean(), df['Longitude'].mean()], zoom_start=11)

# add location markers to map, color codedd accd. to cluster id
for lat, lng, nbh, lbl in zip(df['Latitude'], df['Longitude'], df['Neighbourhood'], df['ClusterLabel']):
    label = folium.Popup(nbh, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=matplotlib.colors.to_hex('C{}'.format(lbl)),
        fill=True,
        fill_color=matplotlib.colors.to_hex('C{}'.format(lbl)),
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map) 

# add centroids for each cluster id
for lbl, (lat, lng) in enumerate(kmeans.cluster_centers_):
    label = folium.Popup('cluster {}'.format(lbl), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=20,
        popup=label,
        color=matplotlib.colors.to_hex('C{}'.format(lbl)),
        fill=False,
        parse_html=False).add_to(toronto_map) 

toronto_map

# Conclusion:
Clustering of the geographic locations of the neighbourhoods using the KMeans++ algorithm into 8 clusters results in one densely packed cluster spanning most of the downtown areas east of the Lower Don River, with larger, more sparse clusters 
for the surrounding, presumably more residential, neighbourhoods of Toronto. 